In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import pandas as pd
import torch
import json
import random

In [6]:
data_dir = "./data/ml-1m"
file_path = os.path.join(data_dir, 'ratings.dat')

names = ['user_id', 'item_id', 'rating', 'timestamp']
df_raw = pd.read_csv(file_path, sep = '::', names = names)

value_col = "rating"
user_col = "user_id"
item_col = "item_id"
print("dimension:", df_raw.shape)
df_raw.head()  

/zf15/rc7ne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


dimension: (1000209, 4)


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [54]:
# title_col = "title"
# genre_col = "genres"

# item_info_path = os.path.join(data_dir, "movies.csv")
# df_item = pd.read_csv(item_info_path)
# df_item = df_item[df_item[genre_col]!="(no genres listed)"]
# print("dimension: ", df_item.shape)
# df_item.head()

In [7]:
df_rating = df_raw[df_raw[value_col] >= 4.0].copy()
# df_rating = df_rating.merge(df_item, on=item_col)

### remove unpopular items and unpopular users

In [8]:
def threshold_user_item(df, uid_min, iid_min):
    n_users = df[user_col].unique().shape[0]
    n_items = df[item_col].unique().shape[0]
    sparsity = float(df.shape[0])/float(n_users*n_items)*100
    print("info")
    print("number of users: {}".format(n_users))
    print("number of items: {}".format(n_items))
    print("sparsity:{:4.4f}%".format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        item_counts = df.groupby(user_col)[item_col].count()
        df = df[~df[user_col].isin(item_counts[item_counts < iid_min].index.tolist())]
        user_counts = df.groupby(item_col)[user_col].count()
        df = df[~df[item_col].isin(user_counts[user_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
            
    n_users = df[user_col].unique().shape[0]
    n_items = df[item_col].unique().shape[0]
    sparsity = float(df.shape[0])/float(n_users*n_items)*100
    print("number of users: {}".format(n_users))
    print("number of items: {}".format(n_items))
    print("sparsity:{:4.4f}%".format(sparsity))
    
    return df

In [9]:
user_min_num = 10
item_min_num = 10
new_df_rating = threshold_user_item(df_rating, user_min_num, item_min_num)

info
number of users: 6038
number of items: 3533
sparsity:2.6968%
number of users: 5949
number of items: 2810
sparsity:3.4189%


In [10]:
movieId_to_iid = {}
movieId_to_iid["<pad>"] = 0
iid_to_movieId = {}
iid_to_movieId[0] = "<pad>"

for (idx, movieId) in enumerate(new_df_rating[item_col].unique().tolist()):
    movieId_to_iid[movieId] = idx+1
    iid_to_movieId[idx+1] = movieId
    
userId_to_uid = {}
userId_to_uid["<pad>"] = 0
uid_to_userId = {}
uid_to_userId[0] = "<pad>"

for (idx, userId) in enumerate(new_df_rating[user_col].unique().tolist()):
    userId_to_uid[userId] = idx+1
    uid_to_userId[idx+1] = userId 

In [11]:
user_num = len(uid_to_userId)
item_num = len(iid_to_movieId)
print("user_num:", user_num)
print("item_num:", item_num)

user_num: 5950
item_num: 2811


In [12]:
new_df_rating["user_id"] = new_df_rating[user_col].apply(lambda x: userId_to_uid[x])
new_df_rating["item_id"] = new_df_rating[item_col].apply(lambda x: movieId_to_iid[x])

In [13]:
user_itemlist_dict = new_df_rating.groupby([user_col]).item_id.apply(list)
user_itemlist_dict = dict(user_itemlist_dict)

In [14]:
train_itemlist = list(new_df_rating[item_col].unique())

In [15]:
def split_train_test(user_data):
    neg_sample = 500
    train = []
    valid = []
    test = []
    
    train_ratio = 0.8
    valid_ratio = 0.9
#     test_ratio = 0.1
    
    userlist = list(user_data.keys())
    usernum = len(userlist)
        
    for i in range(usernum):
        user_i = userlist[i]
        
        itemlist_i = user_data[user_i]
        random.shuffle(itemlist_i)
        itemnum_i = len(itemlist_i)
        
        valid_threshold = int(itemnum_i*train_ratio)
        test_threshold = int(itemnum_i*valid_ratio)
        
        train_itemlist_i = itemlist_i[:valid_threshold]
        valid_itemlist_i = itemlist_i[valid_threshold:test_threshold]
        test_itemlist_i = itemlist_i[test_threshold:]
        
        train_negitemlist_i = set(train_itemlist_i)^set(train_itemlist)
        train_negitemlist_i = list(train_negitemlist_i)
        
        for item_j in train_itemlist_i:
            sampled_negitemlist_i = random.sample(train_negitemlist_i, k=neg_sample)
            train.append([user_i, item_j, sampled_negitemlist_i])
        
        for item_j in valid_itemlist_i:
            valid.append([user_i, item_j])
            
        for item_j in test_itemlist_i:
            test.append([user_i, item_j])
            
    print("train_num", len(train))
    print("valid num", len(valid))
    print("test num", len(test))
    
    return train, valid, test

In [16]:
train_data, valid_data, test_data = split_train_test(user_itemlist_dict)

train_num 454860
valid num 56864
test num 59807


In [17]:
train_df = pd.DataFrame(train_data)
train_df.columns=["userid", "pos_itemid", "neg_itemid"]
valid_df = pd.DataFrame(valid_data)
valid_df.columns=["userid", "itemid"]
test_df = pd.DataFrame(test_data)
test_df.columns=["userid", "itemid"]
print("train num", len(train_df))
print("valid num", len(valid_df))
print("test num", len(test_df))

train num 454860
valid num 56864
test num 59807


In [18]:
train_df.head()

,userid,pos_itemid,neg_itemid
0,1,38,"[2467, 915, 1043, 621, 2731, 707, 2691, 800, 1..."
1,1,19,"[570, 1342, 2311, 1442, 276, 1398, 210, 1308, ..."
2,1,16,"[1633, 188, 336, 1707, 2589, 2671, 948, 223, 2..."
3,1,36,"[2244, 997, 606, 1258, 1509, 2362, 82, 184, 27..."
4,1,17,"[989, 815, 483, 1351, 1781, 2378, 1164, 1246, ..."


In [19]:
train_data_file = "train_data.pickle"
train_data_abs_file = os.path.join(data_dir, train_data_file)
train_df.to_pickle(train_data_abs_file)

valid_data_file = "valid_data.pickle"
valid_data_abs_file = os.path.join(data_dir, valid_data_file)
valid_df.to_pickle(valid_data_abs_file)

test_data_file = "test_data.pickle"
test_data_abs_file = os.path.join(data_dir, test_data_file)
test_df.to_pickle(test_data_abs_file)

In [20]:
userId_to_uid = {str(k):str(userId_to_uid[k]) for k in userId_to_uid}
movieId_to_iid = {str(k):str(movieId_to_iid[k]) for k in movieId_to_iid}

uid_to_userId = {str(k):str(uid_to_userId[k]) for k in uid_to_userId}
iid_to_movieId = {str(k):str(iid_to_movieId[k]) for k in iid_to_movieId}

vocab = {"userindex2uid": userId_to_uid, "uid2userindex":uid_to_userId, "itemindex2iid": movieId_to_iid, "iid2itemindex":iid_to_movieId}
vocab_file = "vocab.json"

with open(os.path.join(data_dir, vocab_file), "w", encoding="utf8") as f:
    json.dump(vocab, f)

In [21]:
data_dir

'./data/ml-1m'

In [78]:
# train_df.userid.unique()
print(max(train_df.pos_itemid.unique()), min(train_df.pos_itemid.unique()))

822 1


In [49]:
a = torch.randn((3, 4))

In [50]:
a

tensor([[ 0.2862, -1.0019,  0.0103,  0.0317],
        [ 0.7306,  0.9805,  1.1846, -1.0710],
        [-1.0851, -0.9494,  0.0846, -1.1893]])

In [51]:
b = torch.tensor([[1, 0, 0], [2, 1, 3], [1, 2, 3]])
a.scatter_(1, b, float("-inf"))

tensor([[   -inf,    -inf,  0.0103,  0.0317],
        [ 0.7306,    -inf,    -inf,    -inf],
        [-1.0851,    -inf,    -inf,    -inf]])

In [52]:
a

tensor([[   -inf,    -inf,  0.0103,  0.0317],
        [ 0.7306,    -inf,    -inf,    -inf],
        [-1.0851,    -inf,    -inf,    -inf]])

In [54]:
torch.topk(a, 1, -1)

torch.return_types.topk(
values=tensor([[ 0.0317],
        [ 0.7306],
        [-1.0851]]),
indices=tensor([[3],
        [0],
        [0]]))

In [55]:
a[:, 0] = float("-inf")

In [56]:
a

tensor([[  -inf,   -inf, 0.0103, 0.0317],
        [  -inf,   -inf,   -inf,   -inf],
        [  -inf,   -inf,   -inf,   -inf]])

In [58]:
list(a[0].numpy() == 0)

[False, False, False, False]

In [20]:
np.random.randint(10)

6